# Treasury Bond Fund Simulator

This is based off of [longinvest's post on bogleheads][1]. He implemented it all in a spreadsheet (which is linked in the thread).

The goal is to calculate returns of a simulated bond fund given a bunch of interest rates. By having returns (instead of rates) we can perform backtesting with historical data.

Keep in mind that this a simulation so it has some simplifying things that will make it slightly different from the real world. This is not a comprehensive list:

* It assumes no transaction costs. This is mostly truish for Treasuries, though.
* It assumes an extremely liquid marketplace where there is no margin when selling. This is mostly truish for Treasuries, as well.
* Because bonds are only rolled once a year, [the duration of the fund shifts a bit][2].
* Real funds [shift their durations all the time][3], even if only by small amounts. Our simulated shifts and their actual shifts are unlikely to line up exactly.
* All interest is reinvested rather than being paid out. This actually works in our favor, since total returns are what we care about for backtesting anyway.

First we need to import some libraries....

[1]: https://www.bogleheads.org/forum/viewtopic.php?f=10&t=179425&start=150
[2]: https://www.bogleheads.org/forum/viewtopic.php?f=10&t=179425&start=100#p2998433
[3]: https://www.bogleheads.org/forum/viewtopic.php?f=10&t=179425&start=100#p2998031

In [1]:
import numpy
from collections import deque
import pandas
import math

# Simulating the Bond Fund
Simulating the bond fund is conceptually straightforward.

You have a ladder of bonds, one for each year. Something like this:

    deque([Maturity: 1 | Yield: 5.00% | Face Value: $50.00,
       Maturity: 2 | Yield: 5.00% | Face Value: $52.50,
       Maturity: 3 | Yield: 5.00% | Face Value: $55.12,
       Maturity: 4 | Yield: 5.00% | Face Value: $57.88,
       Maturity: 5 | Yield: 5.00% | Face Value: $60.78,
       Maturity: 6 | Yield: 5.00% | Face Value: $63.81,
       Maturity: 7 | Yield: 5.00% | Face Value: $67.00,
       Maturity: 8 | Yield: 5.00% | Face Value: $70.36,
       Maturity: 9 | Yield: 5.00% | Face Value: $73.87])
           
Every year three things will happen:

1. All of the bonds will pay out their cash coupon. This is based on their yield and their face value.
1. When a bond gets "too young" (I'll come back to this) we sell it. The exact price will also be explained later. Every year you will sell one bond of the youngest maturity.
1. Now you've got a pile of cash and one fewer bond. Use the cash to buy a new bond of the longest maturity.

## Youngest maturity & oldest maturity

When you create the bond fund, you can select the youngest maturity and the oldest maturity. Say that you want fund where the oldest bond has a 10-year maturity and the youngest bond has a 2-year maturity. As a shorthand, we'll call that a 10-2 fund. Every year a 2-year bond becomes a 1-year bond and will be sold and replaced with a brand new 10-year bond.

In [2]:
def iterate_fund(ladder, yield_curve, max_maturity):
    reduce_maturity(ladder)
    
    payments = get_payments(ladder)

    sold_bond = ladder.popleft()
    payments += sold_bond.value(yield_curve)

    new_bond = Bond(payments, yield_curve[max_maturity-1], max_maturity)
    ladder.append(new_bond)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest does...)
    nav = get_nav(ladder, yield_curve)

    return (ladder, payments, nav)

def get_nav(ladder, rates):
    return sum((b.value(rates) for b in ladder))

def get_payments(ladder):
    return sum((b.gen_payment() for b in ladder))

def reduce_maturity(ladder):
    for b in ladder:
        b.maturity -= 1
    return ladder

# Bond Mechanics

A bond is just three things: a yield, a face value, and a maturity. If you called up your broker you would say, "I want to buy $100 of the 10-year Treasury that is yielding 3.2%." The maturity is 10-years; the face value is $100; and the yield is 3.2%.

There are only two things you can do with a bond.

### Receive your payment
Every year the bond will generate a payment -- a "coupon" in bond-speak. This is simply the **yield × face value**. Going back to the previous example, with a face value of $100 and a yield of 3.2%, every year you would get a payment of $3.20. (Not very impressive, admittedly.)

### Check the current value of the bond
Bonds are designed to be held until their maturity. At that point you'll receive a payment for the face value. In our example, that would mean after holding the bond for 10 years you would get your full $100 back.

But what if you wanted to sell the bond **before** maturity? That's (usually) possible but the exact price will depend on current rates. Say we want to sell our bond after 9 years. In essence, we have a 1-year bond that yields 3.2%. What if the current going yield for 1-year bonds was 2.5%? Then our bond will be worth a little more. If the current going yield for 1-year bonds is 4.2% then our bond will be worth a little less.

Here's how it actually gets calculated:

* take the current maturity remaining on the bond
* take the current yield on bonds of that maturity
* take the bond face value

Then mix all of them into present value calculation: **pv(current yield, current maturity, face value)**. (The pv function is found in every spreadsheet and many calculators.)

If the current yields are 2.5% and you have a face value of $100 then the present value is $97.56. This calculates the "exact" value of the bond. In the real world, someone would probably offer you slightly less, maybe $97.25, because they are offering you liquidity and taking some risk. For Treasuries that is a very small number, so ignoring it is probably good enough.

### From rates to returns
At the end of the day, checking the current value of the bonds we hold is what we're trying to achieve. By adding up the value of all the bonds we hold we can figure out the Net Asset Value (NAV) of our fund. And then we compare that NAV over time. This is what we wanted: to be able to calculate the returns of a (simulated) bond fund.

In [3]:
class Bond:
    def __init__(self, face_value, yield_pct, maturity):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        
    def __repr__(self):
        return ('Maturity: %d | Yield: %.2f%% | Face Value: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))
        
    def gen_payment(self):
        return self.face_value * self.yield_pct
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity, self.gen_payment(), self.face_value)
        return -value

# Bootstrapping the Ladder

Our bond ladder is straightforward enough. Sell the youngest bond and buy another one of the old bonds, using whatever cash we currently have available.

But how do you get the ladder **started**? Where do those first bonds come from?

Here's where things get a little bit unavoidably hacky. In the real world, you could slowly build up a ladder over time. For instance, buy 1/10th of the ladder every year for a decade. That takes, well, a decade. Which means there's an entire decade in our simulation with no results. We can shortcut that at the cost of a slight loss of accuracy for those first few years.

If we're building a 10-2 ladder then we have 9 bonds (we don't have a bond with 1-year maturity, hence only 9 bonds). We bootstrap the ladder by buying all 9 instantly. That means they will all have the same yield -- whatever the current yield is.

In [4]:
def bootstrap(yield_curve, max_bonds, min_maturity):
    bond_yield = yield_curve[max_bonds - 1]
    ladder = deque()
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)

    for i, j in zip(range(max_bonds), range(min_maturity, max_bonds+1)):
        face_value = pow(1 + bond_yield, i) * starting_face_value
        b = Bond(face_value, bond_yield, j)
        ladder.append(b)
    return ladder
bootstrap([.0532]*10, 10, 2)

deque([Maturity: 2 | Yield: 5.32% | Face Value: $50.00,
       Maturity: 3 | Yield: 5.32% | Face Value: $52.66,
       Maturity: 4 | Yield: 5.32% | Face Value: $55.46,
       Maturity: 5 | Yield: 5.32% | Face Value: $58.41,
       Maturity: 6 | Yield: 5.32% | Face Value: $61.52,
       Maturity: 7 | Yield: 5.32% | Face Value: $64.79,
       Maturity: 8 | Yield: 5.32% | Face Value: $68.24,
       Maturity: 9 | Yield: 5.32% | Face Value: $71.87,
       Maturity: 10 | Yield: 5.32% | Face Value: $75.69])

Why do we have a different face value for each one? Why not just $50 for each? That's how we ensure that each rung of the ladder has equivalent value. If they each had a face value of $50 then the longer-term bonds would actually be worth less than the younger-term bonds. [For more explanation, refer to this post by longinvest][1].

[1]: https://www.bogleheads.org/forum/viewtopic.php?p=3142416#p3142416

# Rates

Now that we understand how the ladder works and how to bootstrap it, we need a source of rates in order to drive the engine.

We have a number of sources of rate data.

* Shiller provides 10 year yields on Treasuries, going back to 1871
* Shiller provides 1 year interest rates, going back to 1871
* [FRED provides 1-, 2-, 3-, 5-, 7-, 20-, and 30-year rates][1]. The data begins in the 1954-1977 range. When available, we prefer the FRED data over Shiller data.

So we will start by importing those. (I've spliced them all into a single CSV file to make importing things simpler.)

[1]: https://fred.stlouisfed.org/series/GS1

In [5]:
HISTORICAL_RATES = pandas.read_csv('bond_rates.csv')
HISTORICAL_RATES.head()

,Year,1 year,2 year,3 year,5 year,7 year,10 year,20 year,30 year
0,1871,0.0635,NaN,NaN,NaN,NaN,0.0532,NaN,NaN
1,1872,0.0781,NaN,NaN,NaN,NaN,0.0536,NaN,NaN
2,1873,0.0835,NaN,NaN,NaN,NaN,0.0558,NaN,NaN
3,1874,0.0686,NaN,NaN,NaN,NaN,0.0547,NaN,NaN
4,1875,0.0496,NaN,NaN,NaN,NaN,0.0507,NaN,NaN


## Rate interpolation

For a given year, we will have **some** rate data. At the very least we will have the 1-year and 10-year rates; the data on those go back the further thanks to Shiller.

However, we may *also* have other rate data from FRED.

But we need to have rate data for every year on the yield curve. That is: 1-, 2-, 3-, 4-, 5-, 6-, 7-, 9-, and 10-year rates. When we don't have the data available we will perform linear interpolation from data we *do* have to fill in the gaps.

So if we only have the 1- and 10-year data then we need to do a linear interpolation for the other 8 years. If we have 1-, 3-, and 10-year data then we do linear interpolation between the 1- and 3-year data to fill in the 2-year data. And we'll do linear interpolation between the 3- and 10-year data for the rest.

### Missing data & flat yield curve

We can only do linear interpolation if we have two sets of data. We *always* have data for 1-year and 10-year rates, so we can fill in that part of the yield curve. But before the FRED data series of the mid-20th century we don't have any rates beyond 10 years. How do we fill that part of the rate yield curve when we don't have anything to interpolate?

We don't interpolate: we just create a flat yield curve. That is, the 11-year rate is the same as the 10-year rate. And the 20-year rate is *also* the same as the 10-year rate. And the 30-year rate is **also** the same as the 10-year rate.

This is obviously far from idea. So you should take with a large grain of salt any results before 1954. longinvest has [some comments on the longer terms starting approximately here][2].

### Potential problems with linear interpolation

This linear interpolation is not perfect: it assumes that the yield curve is linear and that may not be the case. In particular, [look at this post from Fryxell][1] where he notes that before the 1920s the yield curve may have looked very different from what it did today.

Still, trying to handle that is beyond the scope of this simulation. The more historical data (like those extra FRED data points) that we have, the less of this linear interpolation we need to do. That makes our post-1954 numbers better than the earlier numbers.

[1]: https://www.bogleheads.org/forum/viewtopic.php?f=10&t=179425&start=100#p2973643
[2]: https://www.bogleheads.org/forum/viewtopic.php?f=10&t=179425&start=100#p3013350

In [6]:
def build_yield_curve(raw_rates, yield_curve_size=30):
    s = pandas.Series(math.nan, index=numpy.arange(yield_curve_size))

    # First try to pre-load any FRED rates.
    # We use NaN to indicate "the data needs to be interpolated"
    s.iloc[0] = raw_rates['1 year']
    s.iloc[1] = raw_rates['2 year']
    s.iloc[2] = raw_rates['3 year']
    s.iloc[4] = raw_rates['5 year']
    s.iloc[6] = raw_rates['7 year']
    s.iloc[9] = raw_rates['10 year']
    s.iloc[19] = raw_rates['20 year']
    s.iloc[20] = raw_rates['30 year']
    
    def left_number(series, index):
        """ Find the index of first number to the left """
        if not math.isnan(series.iloc[index]):
            return index
        else:
            return left_number(series, index-1)
        
    def right_number(series, index):
        """ Find the index of the first number to the right """
        if not math.isnan(series.iloc[index]):
            return index
        else:
            return right_number(series, index+1)
                
    # now fill in the gaps with linear interpolation.
    for i in range(yield_curve_size):
        if math.isnan(s.iloc[i]):
            # First, try to find any existing data on the left and right.
            # We might not find any, for instance when we look beyond 10-years
            # before we have FRED data.

            try:
                left = left_number(s, i)
            except IndexError:
                left = None
                
            try:
                right = right_number(s, i)
            except IndexError:
                right = None

            if (left is None) and (right is None):
                raise IndexError("Couldn't find any rate data to fill out the yield curve.")

            if left is None:
                # If we can't find any data to the left then we can't do any linear interpolation
                # So just fill from the right
                s.iloc[i] = s.iloc[right]
            elif right is None:
                # If we can't find any data to the right then fill from the left
                # Both of these will result in a flat yield curve, which isn't ideal
                s.iloc[i] = s.iloc[left]
            else:
                # We can actually do linear interpolation
                steps = right - left
                rate = s.iloc[left] + ((s.iloc[right] - s.iloc[left]) * (i - left) / steps)
                s.iloc[i] = rate

    return s.tolist()

In [7]:
['%.2f' % (s*100) for s in build_yield_curve(HISTORICAL_RATES.iloc[0])]

['6.35',
 '6.24',
 '6.12',
 '6.01',
 '5.89',
 '5.78',
 '5.66',
 '5.55',
 '5.43',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32',
 '5.32']

# Putting it all together

Now we have all the building blocks. We have a source of rates. We have a way to bootstrap our ladder. We have a way to see how the NAV changes over time.

We only have one decision left to make -- what are the youngest & oldest maturities that we care about? Do we want a 10-2 fund? Or 10-4 fund? Or a 3-2 fund? Or how about a 7-4 fund?

The maximum you can chose is 30 and the minimum is 1.

Do you want a 10-2 fund, or 10-4 fund, or something else? That's actually done by the way you create the bootstrap ladder. This is how you build a 10-4 ladder.

In [8]:
bootstrap(build_yield_curve(HISTORICAL_RATES.iloc[0]), 10, 4)

deque([Maturity: 4 | Yield: 5.32% | Face Value: $50.00,
       Maturity: 5 | Yield: 5.32% | Face Value: $52.66,
       Maturity: 6 | Yield: 5.32% | Face Value: $55.46,
       Maturity: 7 | Yield: 5.32% | Face Value: $58.41,
       Maturity: 8 | Yield: 5.32% | Face Value: $61.52,
       Maturity: 9 | Yield: 5.32% | Face Value: $64.79,
       Maturity: 10 | Yield: 5.32% | Face Value: $68.24])

In [9]:
def loop(ladder, rates, max_maturity):
    # I'm not thrilled about hardcoding 1871 and 2017 in here...
    df = pandas.DataFrame(columns=['NAV', 'Payments', 'Change'], index=numpy.arange(1871, 2017))

    for (i, current_rates) in rates:
        (ladder, payments, nav) = iterate_fund(ladder, build_yield_curve(current_rates), max_maturity)
        df.iloc[i] = {'NAV' : nav, 'Payments' : payments}

    calculate_returns(df)
    return df

def calculate_returns(df):
    # Longinvest calculates the return based on comparison's to
    # next year's NAV. So I'll do the same. Even though that seems
    # weird to me. Maybe it's because the rates are based on January?
    # Hmmm...that sounds plausible.
    max_row = df.shape[0]

    for i in range(max_row - 1):
        next_nav = df.iloc[i+1]['NAV']
        nav = df.iloc[i]['NAV']
        change = (next_nav - nav) / nav
        df.iloc[i]['Change'] = change
    return df

def simulate(max_maturity, min_maturity, rates):
    """ This is just something to save on typing...and make clearer what the bounds on the fund are """
    ladder = bootstrap(build_yield_curve(rates.iloc[0]), max_maturity, min_maturity)
    return loop(ladder, rates.iterrows(), max_maturity)

# Simulate All The Things

Now we can simulate a 10-2 fund.

In [10]:
simulate(10, 2, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,578.504,79.2358,0.026633
1872,593.912,82.7192,0.0367096
1873,615.714,86.8182,0.0821551
1874,666.298,92.3718,0.097946
1875,731.559,98.4774,0.0574012


Or a 10-4 fund.

In [11]:
simulate(10, 4, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,425.293,70.8014,0.0232979
1872,435.202,72.9607,0.0308817
1873,448.641,76.0795,0.0796149
1874,484.36,81.8895,0.0998552
1875,532.726,88.834,0.0600275


In [12]:
simulate(4, 2, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,168.198,59.4006,0.0364836
1872,174.334,62.2434,0.0586692
1873,184.562,66.2677,0.0952411
1874,202.14,71.7686,0.0939793
1875,221.137,77.3144,0.0455889


In [13]:
simulate(3, 2, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,109.15,56.2008,0.041584
1872,113.689,58.9174,0.0631368
1873,120.867,62.7655,0.0938777
1874,132.214,68.2762,0.0893958
1875,144.033,73.752,0.045652


# Holding to maturity

Our previous examples all sell prior to bond maturity.

If you want to hold until maturity, then specify a minimum duration of 1.

In [14]:
simulate(10, 1, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,661.941,83.9612,0.0295413
1872,681.495,88.4279,0.0399521
1873,708.723,93.1676,0.0821584
1874,766.95,98.3664,0.0956076
1875,840.276,103.747,0.0568201


# Only holding a single maturity

If you only want to fund to hold a single maturity, specify the same number twice. This constructs fund that only holds bonds of 2-year maturity. That is, every year it sells them and buys new 2-year bonds.

In [15]:
simulate(2, 2, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,53.064,53.064,0.0477517
1872,55.5979,55.5979,0.0678815
1873,59.3719,59.3719,0.0914855
1874,64.8036,64.8036,0.0836862
1875,70.2268,70.2268,0.0463255


# Going beyond 30 years

Keeping in mind the caveats above about results before we have FRED data from the mid-1950s...we can also build longer term bond funds.

In [16]:
simulate(30, 2, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,3450.53,224.31,0.0446789
1872,3604.7,235.511,0.0287824
1873,3708.45,247.8,0.071365
1874,3973.1,262.336,0.103874
1875,4385.8,277.739,0.103619


# Crazy Funds

You can select any set of maturities you want, since the computer does all the heavy lifting. Want to simulate a fund that is **28-14** or **17-9**? That's easy.

In [17]:
simulate(28, 14, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,1164.08,108.801,0.0482991
1872,1220.31,114.396,0.0271181
1873,1253.4,119.412,0.0692703
1874,1340.22,126.584,0.105206
1875,1481.22,135.747,0.113024


In [18]:
simulate(17, 9, HISTORICAL_RATES).head()

,NAV,Payments,Change
1871,587.224,78.9965,0.0456297
1872,614.019,82.0043,0.0309193
1873,633.004,85.3259,0.0646905
1874,673.953,91.4001,0.0917573
1875,735.794,99.1681,0.0927011
